# XOR

In [15]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json, matplotlib
s = json.load( open("styles/bmh_matplotlibrc.json") )
matplotlib.rcParams.update(s)
from IPython.core.pylabtools import figsize
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]

In [16]:
from mpl_toolkits.mplot3d import Axes3D

In [17]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [18]:
import numpy as np

## Función de activación

In [19]:
def logistica(z):
    """
    Devuelve la función logística evaluada componente por componente
    """
    return 1 / (1 + np.exp(-z))

In [20]:
from matplotlib.pyplot import figure

@interact
def plot_log():
    z = np.arange(-5,5,0.1)
    figure(figsize=(4,2))
    plt.plot(z, logistica(z))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma$")
    plt.title("$\sigma = \\frac{1}{1 + e^{-z}}$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [21]:
def derivada_logistica(z):
    """
    Función que, dado un arreglo de valores z
    calcula el valor de la derivada para cada entrada.
    """
    g = logistica(z)
    return g * (1 - g)

In [22]:
@interact
def plot_logp():
    z = np.arange(-5,5,0.1)
    figure(figsize=(4,2))
    plt.plot(z, derivada_logistica(z))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma'$")
    plt.title("$\sigma' = \sigma (1-\sigma)$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [23]:
def derivada_logistica_atajo(val_sigma):
    """
    Función que, dado un arreglo de valores de sigma(z)
    calcula el valor de la derivada para cada entrada.
    
    Si ya se cuenta con esos valores, es más eficiente
    calcular esto directamente.
    """
    return val_sigma * (1 - val_sigma)

$\frac{d\sigma}{dz} = \sigma(z) \cdot (1 - \sigma(z))$

In [24]:
@interact
def plot_logpa():
    z = np.arange(-5,5,0.1)
    val = logistica(z)
    figure(figsize=(4,2))
    plt.plot(z, derivada_logistica_atajo(val))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma'$")
    plt.title("$\sigma' = \sigma (1-\sigma)$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [25]:
np.random.seed(10)

## Función de error: Entropía cruzada

La función de error más sencilla utilizada para problemas de clasificación es la **entropía cruzada**
\begin{align}
  J(\Theta) =& - \frac{1}{m} \left[ \sum_{i=1}^m \sum_{k=1}^K   y_k^{(i)} \log(h_\Theta(x^{(i)}))_k  + (1 - y_k^{(i)}) \log(1 - h_\Theta(x^{(i)}))_k   \right]    +  \\
  & \frac{\lambda}{2m} \sum_{l=1}^{L-1} \sum_{i=1}^{s_L} \sum_{j=1}^{s_{l+1}} (\theta_{ji}^{(l)})^2
\end{align}
donde:
  * $K$ es el número de neuronas de salida.
  * $s_l$ es el número de neuronas en la capa $l$.
  * $m$ es el número de ejemplares de entrenamiento.
  
En su forma vectorizada, las componentes del gradiente están dadas por:
\begin{align}
  \delta^{(L)} &= (A^{(L)})^T - Y  \\
  \delta^{(L-1)} &= \delta^{(L)} \Theta_{[:,1:]}^{(L-1)} \circ g'(z^{(L-1)}) \\
  ... \\
  \delta^{(1)} &= \delta^{(0)} \Theta_{[:,1:]}^{(1)} \circ g'(z^{(1)}) \\
\end{align}
con:
\begin{align}
  g'(z^{(l)}) = A^{(l)} \circ (1 - A^{(l)})
\end{align}
en general:
\begin{align}
  \Delta^{(l)} =& (\delta^{(l+1)})^T A^{(l)}   &   \nabla^{(l)} =& \frac{1}{m}\Delta^{(l)}
\end{align}

In [26]:
def cross_entropy(y, h):
    return - y * np.log(h) - (1 - y) * np.log(1 - h)

In [30]:
@interact
def plot_crossentropy():
    """
    Muestra como, en el rango donde está definida, la entropía cruzada tiene
    sus valores más pequeños donde 'y' y 'h' coinciden, y su valor más alto
    donde éstos son opuestos.
    """
    fig = plt.figure()

    #ax = fig.gca(projection='3d') # Metodo 1
    ax = fig.add_subplot(111, projection='3d') # Metodo 2
    # Cambio al metodo 2 porque el 1 no funciona en mi compu 
    
    # Datos
    X = np.arange(0.01, 1, 0.05)
    Y = np.arange(0.01, 1, 0.05)
    X, Y = np.meshgrid(X, Y)
    CE = cross_entropy(X, Y)
    
    surf = ax.plot_surface(X, Y, CE, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    
    ax.set_xlabel("$y$")
    ax.set_ylabel("$h$")
    ax.set_title("Entropía cruzada")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

## Red neuronal
La red implementa encadenamiento hacia adelante (para evaluar) y hacia atrás (para entrenarse).

In [31]:
# Cuando se programa un algoritmo que depende de la generación de números aleatorios
# es buena costumbre fijar la semilla a partir de la cual dichos números son generados.
# De este modo el comportamiento sobre los datos de entrada será predecible mientras
# se está desarrollando/probando el código.
#
# Una vez el código está listo, se debe usar variando los valores de esta semilla
# para ver los efectos de la aleatoriedad simulada.

np.random.seed(10)

In [15]:
class Multicapa:
    """
    Red neuronal con tres capas:
    
    Entrada
    Oculta
    Salida
    
    Los parámetros (pesos) que conectan las capas se encuentran en matrices
    con los nombres siguientes:
    
    Entrada -> self.Theta_0 -> Oculta
    Oculta  -> self.Theta_1 -> Salida
    """
    def __init__(self, n_entrada, n_ocultas, n_salidas):
        """
        Inicializa la red neuronal, con pesos Theta_0 y Theta_1 aleatorios,
        esta implementación debe incluir el uso de sesgos, por lo que éstos
        no se cuentan en los parámetros siguientes, puedes incluirlos como
        neuronas extra o en sus propias matrices, sólo sé consistente pues
        esto afectará tu implementación.
        
        :param n_entrada: número de datos de entrada (sin contar el sesgo)
        :param n_ocultas: número de neuronas ocultas
        :param n_salidas: número de nueronas de salida
        """
        ## TODO
    
    def feed_forward(self, X, vector = None):
        """ Calcula las salidas, dados los datos de entrada en forma de matriz.
        Guarda los parámetros siguientes:
        A0: activaciones de la capa de entrada, ya con sesgos
        Z1: potenciales de la capa oculta, aún sin sesgo
        A1: activaciones de la capa oculta, ya con sesgos
        Z2: potenciales de la capa de salida
        A2: activaciones de la capa de salida
        
        :param vector: [opcional] se utilizarán los pesos indicados en este
                       vector en lugar de los pesos actuales de la red.
        """
        ## TODO
        
    def back_propagate(self, X, Y, lambda_r = 0.0):
        """ Calcula el error y su gradiente dados los pesos actuales de la red
        y los resultados esperados.
        
        Guarda el error en el atributo self.error y el gradiente en matrices
        self.Grad_1 y self.Grad_0, que tienen la misma forma de Theta_0 y Theta_1.
        
        :param X: matriz de entradas
        :param Y: matriz de salidas deseadas
        :param lambda_r: coeficiente de regularización
        """
        ## TODO
        
    def calc_error(self, X, Y, vector, lambda_r = 0.0):
        """
        Calcula el error que se cometería utilizando los pesos en 'vector' en lugar
        de los pesos actuales de la red.
        
        :returns: el error
        """
        ## TODO
    
    def vector_weights(self):
        """
        Acomoda a todos los parámetros en las matrices de sesgos y pesos, en un solo vector.
        
        :returns: vector de parámetros
        """
        ## TODO
    
    def reconstruct_matrices(self, vector):
        """
        Dado un vector, rearma matrices del tamaño de las matrices de sesgos y pesos.
        
        :returns: matrices de parámetros
        """
        ## TODO
        
    def approx_gradient(self, X, Y, lambda_r = 0.0):
        """
        Aproxima el valor del gradiente alrededor de los pesos actuales,
        perturbando cada peso, uno por uno hasta estimar la variación alrededor
        de cada peso.
        
        En este método se itera sobre cada peso w:
        * Sea w - epsilon -> val1, se calcula el error e1 cometido por la red si w es
                                   reemplazado por val1.
        * Sea w + epsilon -> val2, se calcula el error e2 cometido por la red si w es
                                   reemplazado por val2.
        * La parcial correspondiente se estima como (val1 - val2)/(2 * epsilon)
        
        Este método sólo se utiliza para verificar que backpropagation esté bien
        implementado, ya que en la práctica es muy lento y menos preciso.
        
        :returns: matrices que tienen la misma forma de Theta_0 y Theta_1, donde
                  cada entrada es la estimación de la parcial del error con
                  respecto al peso correspondiente
        """
        ## TODO
        
    def gradient_descent(self, X, Y, alpha, ciclos=10, check_gradient = False, lambda_r = 0.0):
        """ Evalúa y ajusta los pesos de la red, de acuerdo a los datos en X y los resultados
        deseados en Y.  Al final grafica el error vs ciclo.  Si el entrenamiento es correcto
        el error debe descender por cada iteración (ciclo).
        
        :param X: datos de entrada
        :param Y: salidas deseadas
        :param alpha: taza de aprendizaje
        :param ciclos: número de veces que se realizarán ajustes para todo el conjunto de datos X
        :param check_gradient: se calculará el gradiente con backpropagation y con aproximación por
                               perturbaciones, imprimiendo los valores lado a lado para que puedan
                               ser comparados.
        :param lambda_r: coeficiente de regularización
        """
        ## TODO
        
    def print_output(self):
        """
        Muestra en pantalla los valores de salida obtenidos en la última ejecución de feed_forward.
        """
        ## TODO

# Conjunto de datos

Probaremos esta red con la función más sencilla que requiere más de un perceptrón: $XOR$.

Para ello requerimos una red con la arquitectura siguiente:

<img src="figuras/xor_sin_pesos.png">

In [16]:
# Datos
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # Entradas
Y = np.array([[0], [1], [1], [0]])              # Salidas deseadas

# Instanciación de la red
xor = Multicapa(2, 2, 1)

# Probamos qué valores calcula con una inicialización aleatoria
xor.feed_forward(X)
xor.print_output()

[[0.         0.         0.69697434]
 [0.         1.         0.71737828]
 [1.         0.         0.70117284]
 [1.         1.         0.72086962]]


Para verificar si la red funciona para lo que fue diseñada, no basta con reducir el error,
debemos evaluar si calcula la función que queremos que calcule.  En este caso, queremos
usarla para evaluar una función binaria.  Si realizamos un redondeo, veamos cuántas respuestas
calcula correctamente.

In [17]:
def count_correct(red, X, Y):
    """ Conciderando que las etiquetas en Y son binarias (0 y 1)
    la salida h de la red se tomará como 0 si h < 0.5
    y como 1 si h >= 0.5
    """
    red.feed_forward(X)
    H = red.A2.T                         # Ojo: dependiendo de tu implementación, podrías no querer la .T
    H = np.rint(H)
    print("H = ", H)
    print("Y = ", Y)
    c = np.count_nonzero((H - Y)==0)
    print("Se calcularon correctamente ", c, "entradas.")

In [18]:
c = count_correct(xor, X, Y)

H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


Para poder implementar el algoritmo de optimización, las variables a optimizar, es decir, los pesos, deben ser colocados sobre un vector (o matriz columna, en este caso).  Es necesario convertir las matrices de pesos a vector y, para usar la red, revertir el proceso, reconstruyendo las matrices a partir del vector con los pesos.

In [19]:
# Con esta casilla verifica que tus conversiones sean correctas

print("Valores actuales de los pesos")

print("Theta_0:\n", xor.Theta_0, "\n\nTheta_1:\n", xor.Theta_1)

print("\nMatrices a vector de pesos: ")
print(xor.vector_weights())

print("\nReconstrucción de las matrices a partir del vector de pesos: ")
T0, T1 = xor.reconstruct_matrices(xor.vector_weights())
print(T0, T1)

Valores actuales de los pesos
Theta_0:
 [[0.77132064 0.02075195 0.63364823]
 [0.74880388 0.49850701 0.22479665]] 

Theta_1:
 [[0.19806286 0.76053071 0.16911084]]

Matrices a vector de pesos: 
[[0.77132064]
 [0.02075195]
 [0.63364823]
 [0.74880388]
 [0.49850701]
 [0.22479665]
 [0.19806286]
 [0.76053071]
 [0.16911084]]

Reconstrucción de las matrices a partir del vector de pesos: 
[[0.77132064 0.02075195 0.63364823]
 [0.74880388 0.49850701 0.22479665]] [[0.19806286 0.76053071 0.16911084]]


In [20]:
# Con esta casilla revisa que backpropagation y aproximación por perturbaciones
# den resultados semejantes.  Observa que se ejecuta sobre un solo ciclo pues es lento.

xor.gradient_descent(X, Y, 0.3, 1, check_gradient = True)

Grad 0 =  [[0.0293725  0.00922161 0.01291035]
 [0.00648158 0.00243598 0.00224845]] 
Approx =  [[0.0293725  0.00922161 0.01291035]
 [0.00648158 0.00243598 0.00224845]] 
Diff =  [[ 1.68121350e-11 -3.56208986e-12  2.38570969e-12]
 [ 2.19155423e-12 -4.13290496e-13 -1.36334607e-12]] 
Grad 1 =  [[0.20909877 0.15629306 0.15556282]] 
Approx =  [[0.20909877 0.15629306 0.15556282]] 
Diff =  [[1.43075829e-10 6.12830897e-11 6.26605989e-11]]


In [21]:
@interact_manual(ciclos = (50, 2000))
def train_XOR(ciclos):
    xor.gradient_descent(X, Y, 0.3, ciclos)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [22]:
# Después de haber sido correctamente entrenada, la red debe producir salidas
# cercanas a las deseadas.  Dado que la función de activación es la sigmoide
# nunca llegará a 0 o 1 exactamente.

xor.feed_forward(X)
xor.print_output()

print("\nTheta_0 = ", xor.Theta_0, "\n\nTheta_1", xor.Theta_1)

[[0.         0.         0.0953622 ]
 [0.         1.         0.86271536]
 [1.         0.         0.86365152]
 [1.         1.         0.20019896]]

Theta_0 =  [[-1.95332339  5.82298199  5.45721343]
 [-3.69098754  2.53592719  2.49261141]] 

Theta_1 [[-2.88653165  6.31829974 -6.08046539]]


In [23]:
c = count_correct(xor, X, Y)

H =  [[0.]
 [1.]
 [1.]
 [0.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  4 entradas.


## Regularización

In [24]:
xor_reg = Multicapa(2, 2, 1)

In [25]:
c = count_correct(xor_reg, X, Y)

H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


In [26]:
xor.gradient_descent(X, Y, 0.3, 1, check_gradient = True, lambda_r = 0.15)

Grad 0 =  [[0.00595249 0.21407793 0.19852612]
 [0.02135489 0.08273774 0.0805119 ]] 
Approx =  [[0.00595249 0.21407793 0.19852612]
 [0.02135489 0.08273774 0.0805119 ]] 
Diff =  [[-3.59733354e-11  8.05347455e-12  1.34698086e-11]
 [-1.10373738e-10  2.06702988e-11  2.38678244e-11]] 
Grad 1 =  [[ 0.00548201  0.22323298 -0.20391396]] 
Approx =  [[ 0.00548201  0.22323298 -0.20391396]] 
Diff =  [[ 2.73016332e-12  2.47019349e-11 -1.99884553e-11]]


In [27]:
@interact_manual(ciclos = (50, 2000))
def train_XOR_reg(ciclos):
    # Prueba diferentes valores de lambda_r ¿qué tanto lo puedes incrementar si matar a la red?
    xor_reg.gradient_descent(X, Y, 0.3, ciclos, lambda_r = 0.005)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [28]:
xor_reg.feed_forward(X)
xor_reg.print_output()
print("Theta_0 = ", xor_reg.Theta_0, "\nTheta_1", xor_reg.Theta_1)

[[0.         0.         0.20947461]
 [0.         1.         0.60845306]
 [1.         0.         0.63578778]
 [1.         1.         0.57088641]]
Theta_0 =  [[-0.78528765  3.61203011  3.67129172]
 [-0.73680114  0.76181636  1.01854523]] 
Theta_1 [[-1.79868704  3.56551239 -1.99583647]]


In [29]:
c = count_correct(xor_reg, X, Y)

H =  [[0.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  3 entradas.


In [30]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()